In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import pickle

In [2]:
train_df = pd.read_csv('/kaggle/input/disease-prediction-using-machine-learning/Training.csv')
test_df = pd.read_csv('/kaggle/input/disease-prediction-using-machine-learning/Testing.csv')

In [3]:
train_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,Unnamed: 133
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN


In [4]:
cols = train_df.columns
for i in range(len(cols)):
    col = str(cols[i]).replace('_', ' ')
    print(f"'{col}': {i}", end=',')

'itching': 0,'skin rash': 1,'nodal skin eruptions': 2,'continuous sneezing': 3,'shivering': 4,'chills': 5,'joint pain': 6,'stomach pain': 7,'acidity': 8,'ulcers on tongue': 9,'muscle wasting': 10,'vomiting': 11,'burning micturition': 12,'spotting  urination': 13,'fatigue': 14,'weight gain': 15,'anxiety': 16,'cold hands and feets': 17,'mood swings': 18,'weight loss': 19,'restlessness': 20,'lethargy': 21,'patches in throat': 22,'irregular sugar level': 23,'cough': 24,'high fever': 25,'sunken eyes': 26,'breathlessness': 27,'sweating': 28,'dehydration': 29,'indigestion': 30,'headache': 31,'yellowish skin': 32,'dark urine': 33,'nausea': 34,'loss of appetite': 35,'pain behind the eyes': 36,'back pain': 37,'constipation': 38,'abdominal pain': 39,'diarrhoea': 40,'mild fever': 41,'yellow urine': 42,'yellowing of eyes': 43,'acute liver failure': 44,'fluid overload': 45,'swelling of stomach': 46,'swelled lymph nodes': 47,'malaise': 48,'blurred and distorted vision': 49,'phlegm': 50,'throat irrita

In [5]:
cols = train_df.columns
for i in range(len(cols)):
    col = str(cols[i]).replace('_', ' ')
    print(f"{i}: '{col}'", end=',')

0: 'itching',1: 'skin rash',2: 'nodal skin eruptions',3: 'continuous sneezing',4: 'shivering',5: 'chills',6: 'joint pain',7: 'stomach pain',8: 'acidity',9: 'ulcers on tongue',10: 'muscle wasting',11: 'vomiting',12: 'burning micturition',13: 'spotting  urination',14: 'fatigue',15: 'weight gain',16: 'anxiety',17: 'cold hands and feets',18: 'mood swings',19: 'weight loss',20: 'restlessness',21: 'lethargy',22: 'patches in throat',23: 'irregular sugar level',24: 'cough',25: 'high fever',26: 'sunken eyes',27: 'breathlessness',28: 'sweating',29: 'dehydration',30: 'indigestion',31: 'headache',32: 'yellowish skin',33: 'dark urine',34: 'nausea',35: 'loss of appetite',36: 'pain behind the eyes',37: 'back pain',38: 'constipation',39: 'abdominal pain',40: 'diarrhoea',41: 'mild fever',42: 'yellow urine',43: 'yellowing of eyes',44: 'acute liver failure',45: 'fluid overload',46: 'swelling of stomach',47: 'swelled lymph nodes',48: 'malaise',49: 'blurred and distorted vision',50: 'phlegm',51: 'throat ir

In [6]:
train_df.shape

(4920, 134)

In [7]:
test_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [8]:
test_df.shape

(42, 133)

In [9]:
null_check = train_df.isnull().sum()
null_check[null_check != 0]

Unnamed: 133    4920
dtype: int64

In [10]:
train_df = train_df.drop(columns = 'Unnamed: 133')

In [11]:
prognosis = train_df['prognosis']

In [12]:
prognosis.value_counts()

prognosis
Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
All

# Splitting Dataset

In [13]:
train_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [14]:
X_train = train_df.drop(columns='prognosis')
X_test = test_df.drop(columns='prognosis')

y_train = train_df['prognosis'].to_numpy()
y_test = test_df['prognosis']

In [15]:
X_train.shape, X_test.shape

((4920, 132), (42, 132))

In [16]:
y_train.shape, y_test.shape

((4920,), (42,))

# Encdoding

In [17]:
y_train_encoded = pd.get_dummies(y_train)
y_train_encoded.head()

,(vertigo) Paroymsal Positional Vertigo,AIDS,Acne,Alcoholic hepatitis,Allergy,Arthritis,Bronchial Asthma,Cervical spondylosis,Chicken pox,Chronic cholestasis,...,Osteoarthristis,Paralysis (brain hemorrhage),Peptic ulcer diseae,Pneumonia,Psoriasis,Tuberculosis,Typhoid,Urinary tract infection,Varicose veins,hepatitis A
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
y_test_encoded = pd.get_dummies(y_test)
y_test_encoded.head()

,(vertigo) Paroymsal Positional Vertigo,AIDS,Acne,Alcoholic hepatitis,Allergy,Arthritis,Bronchial Asthma,Cervical spondylosis,Chicken pox,Chronic cholestasis,...,Osteoarthristis,Paralysis (brain hemorrhage),Peptic ulcer diseae,Pneumonia,Psoriasis,Tuberculosis,Typhoid,Urinary tract infection,Varicose veins,hepatitis A
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Model

In [19]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape= (X_train.shape[1], )))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_train_encoded.shape[1], activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                4256      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 41)                697       
                                                                 
Total params: 5481 (21.41 KB)
Trainable params: 5481 (21.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
early_stopping= EarlyStopping(patience=2, monitor='val_accuracy')
model.fit(X_train, y_train_encoded, batch_size=40, epochs=10, validation_split=0.25, callbacks=[early_stopping])

Epoch 1/10
93/93 [==============================] - 2s 6ms/step - loss: 3.4615 - accuracy: 0.1474 - val_loss: 2.9804 - val_accuracy: 0.3171
Epoch 2/10
93/93 [==============================] - 0s 3ms/step - loss: 2.2736 - accuracy: 0.5913 - val_loss: 1.2769 - val_accuracy: 0.9024
Epoch 3/10
93/93 [==============================] - 0s 3ms/step - loss: 0.8028 - accuracy: 0.9564 - val_loss: 0.2745 - val_accuracy: 1.0000
Epoch 4/10
93/93 [==============================] - 0s 3ms/step - loss: 0.2203 - accuracy: 0.9995 - val_loss: 0.0701 - val_accuracy: 1.0000
Epoch 5/10
93/93 [==============================] - 0s 3ms/step - loss: 0.0828 - accuracy: 1.0000 - val_loss: 0.0275 - val_accuracy: 1.0000


In [23]:
model.evaluate(X_test, y_test_encoded, batch_size=1, steps=8)

8/8 [==============================] - 0s 2ms/step - loss: 0.0215 - accuracy: 1.0000


[0.021489158272743225, 1.0]

In [24]:
predictions = model.predict(X_test)

2/2 [==============================] - 0s 4ms/step


In [25]:
np.argmax(predictions, axis=1)

array([15,  4, 16,  9, 14, 33,  1, 12, 17,  6, 23, 30,  7, 32, 28, 29,  8,
       11, 37, 40, 19, 20, 21, 22,  3, 36, 10, 34, 13, 18, 39, 26, 24, 25,
       31,  5,  0,  2, 38, 35, 27, 14])

In [26]:
# model.save('model.h5')
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

import shutil
shutil.rmtree("/kaggle/working/model")

In [27]:
def get_result(arr):
    names = ['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes ', 'Dimorphic hemmorhoids(piles)',
       'Drug Reaction', 'Fungal infection', 'GERD', 'Gastroenteritis',
       'Heart attack', 'Hepatitis B', 'Hepatitis C', 'Hepatitis D',
       'Hepatitis E', 'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins', 'hepatitis A']
    
    
    arr = np.array(arr).reshape(1,132)
    prediction = model.predict(arr)
    index = np.argmax(prediction)
    
    disease_name = names[index]
    probability = np.max(model.predict(arr))
    
    return disease_name, probability

In [28]:
X_test.iloc[4].to_numpy()

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
get_result(X_test.iloc[4].to_numpy())

1/1 [==============================] - 0s 25ms/step


('Drug Reaction', 0.97152746)

In [30]:
array_data = y_test_encoded.to_numpy()

for index, row in enumerate(array_data):
    column_name = y_test_encoded.columns[np.where(row == 1)[0][0]]
    print(f"{index}: {column_name}")

0: Fungal infection
1: Allergy
2: GERD
3: Chronic cholestasis
4: Drug Reaction
5: Peptic ulcer diseae
6: AIDS
7: Diabetes 
8: Gastroenteritis
9: Bronchial Asthma
10: Hypertension 
11: Migraine
12: Cervical spondylosis
13: Paralysis (brain hemorrhage)
14: Jaundice
15: Malaria
16: Chicken pox
17: Dengue
18: Typhoid
19: hepatitis A
20: Hepatitis B
21: Hepatitis C
22: Hepatitis D
23: Hepatitis E
24: Alcoholic hepatitis
25: Tuberculosis
26: Common Cold
27: Pneumonia
28: Dimorphic hemmorhoids(piles)
29: Heart attack
30: Varicose veins
31: Hypothyroidism
32: Hyperthyroidism
33: Hypoglycemia
34: Osteoarthristis
35: Arthritis
36: (vertigo) Paroymsal  Positional Vertigo
37: Acne
38: Urinary tract infection
39: Psoriasis
40: Impetigo
41: Fungal infection
